<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myXH_Return_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import yfinance as yf
import pandas as pd

def calculate_stock_gains(start_date, end_date, stock_list):
    """
    Calculate percentage gains for a list of NSE stocks over a specified date range.

    Args:
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.
        stock_list (list): List of NSE stock symbols (e.g., ['RELIANCE.NS', 'TCS.NS']).

    Returns:
        pd.DataFrame: DataFrame containing stock symbols and their percentage gains.
    """
    data = []

    for stock in stock_list:
        try:
            # Fetch stock data for the given date range
            stock_data = yf.Ticker(stock + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            stock_data = stock_data[(stock_data.index <= end_date) & (stock_data.index >= start_date)]

            if not stock_data.empty:
                # Calculate percentage gain
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percentage_gain = ((end_price - start_price) / start_price) * 100

                data.append({
                    'Symbol': stock,
                    'Start Price': start_price,
                    'End Price': end_price,
                    'Gains (%)': round(percentage_gain, 2)
                })
            else:
                data.append({
                    'Symbol': stock,
                    'Start Price': None,
                    'End Price': None,
                    'Gains (%)': None
                })
        except Exception as e:
            data.append({
                'Symbol': stock,
                'Start Price': None,
                'End Price': None,
                'Gains (%)': None,
                'Error': str(e)
            })

    # Create and return DataFrame
    return pd.DataFrame(data)

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# Example usage:
if __name__ == "__main__":
    start_date = "2024-09-27"
    end_date = "2025-04-22"


    pps_df = get_mypps_df()
    scr_df = get_myscreen_df()

    df = pd.merge(pps_df, scr_df, on = 'Symbol')

    #stock_list = df[(df['Conviction'].isin(['X','H'])) & ((df['CapType'] == 'LC'))]['Symbol'].values

    list_x40 = list(pps_df[pps_df['MBQ'].str.contains('X40', na=False)]['Symbol'].values)
    list_x5k = list(pps_df[pps_df['MBQ'].str.contains('X5K', na=False)]['Symbol'].values)
    list_x40n = list(pps_df[pps_df['MBQ'].str.contains('X40N', na=False)]['Symbol'].values)
    #len(set(list_x40 + list_x5k) - set( list_x40n))
    stock_list = list(set(list_x40 + list_x5k) - set( list_x40n))

    #stock_list = ["RELIANCE", "TCS", "INFY", "HDFCBANK"]

    result_df = calculate_stock_gains(start_date, end_date, stock_list)
    result_df = pd.merge(result_df, pps_df, on = 'Symbol')
    cols = ['Symbol', 'Conviction', 'MBQ', 'Start Price', 'End Price', 'Gains (%)']
    result_df = result_df[cols]
    print(str(round(result_df['Gains (%)'].mean())) + '%')

result_df.sort_values(by = 'Gains (%)', ascending=False)

-9%


,Symbol,Conviction,MBQ,Start Price,End Price,Gains (%)
46,KOTAKBANK,X,X40/X5K,1872.449951,2268.800049,21.17
1,BAJFINANCE,X,X40/X5K,7756.000000,9260.500000,19.40
61,BAJAJHLDNG,H,X40/X5K,10543.450195,12324.000000,16.89
49,HDFCBANK,X,X40/X5K,1752.650024,1961.699951,11.93
40,INDIGO,M,AR/X5K,4945.700195,5534.000000,11.90
...,...,...,...,...,...,...
17,COLPAL,X,X40,3734.585938,2658.800049,-28.81
72,BAJAJ-AUTO,X,X40/X5K,12666.400391,8141.000000,-35.73
0,TATAMOTORS,M,XY24/X5K,993.000000,630.849976,-36.47
54,INDUSINDBK,H,XR/X5K,1462.699951,787.500000,-46.16


In [4]:
result_df.sort_values(by = 'Gains (%)', ascending=False).tail(20)

,Symbol,Conviction,MBQ,Start Price,End Price,Gains (%)
51,HINDALCO,M,X5K,747.150024,620.099976,-17.00
36,ICICIGI,H,X40,2231.768799,1845.199951,-17.32
7,HCLTECH,H,X40/X5K,1790.614502,1479.900024,-17.35
42,HAVELLS,X,X40,2026.146240,1664.699951,-17.84
5,VEDL,L,X5K,503.785065,413.049988,-18.01
52,COALINDIA,L,X5K,490.524170,398.600006,-18.74
8,HINDUNILVR,X,XY25/X5K/X40,2954.485352,2398.699951,-18.81
6,LICI,H,XY25/X5K,1024.250000,821.400024,-19.80
53,TCS,X,X40/X5K,4230.771973,3314.399902,-21.66
24,DABUR,X,XY24/X40,629.743225,484.399994,-23.08
